# Title


If you are not familiar with PyMC, you can [start with this chapter from *Think Bayes*](https://allendowney.github.io/ThinkBayes2/chap19.html), especially the World Cup Problem. Or you can [run that chapter on Colab](https://colab.research.google.com/github/AllenDowney/ThinkBayes2/blob/master/notebooks/chap19_v3.ipynb).

You can read [the slides I used to present this example](COMING SOON).

[Click here to run this notebook on Colab](https://colab.research.google.com/github/AllenDowney/SurveyDataPyMC/blob/main/notebooks/01_tutorial.ipynb)

In [1]:
# Get utils.py

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
        
download('https://github.com/AllenDowney/SurveyDataPyMC/raw/main/notebooks/utils.py')

In [2]:
try:
    import empiricaldist
except ImportError:
    !pip install empiricaldist

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm

from utils import decorate, value_counts

In [4]:
# Make the figures smaller to save some screen real estate

plt.rcParams['figure.dpi'] = 75
plt.rcParams['figure.figsize'] = [6, 3.5]
plt.rcParams['axes.titlelocation'] = 'left'

In [5]:
# Get the GSS data

# This dataset is prepared in GssExtract/notebooks/02_make_extract-2022_3a.ipynb

DATA_PATH = "https://github.com/AllenDowney/GssExtract/raw/main/data/interim/"
filename = "gss_extract_2022_3a.hdf"
download(DATA_PATH + filename)

In [6]:
gss = pd.read_hdf(filename, "gss")
gss.shape

(72390, 57)

In [7]:
# https://gssdataexplorer.norc.org/variables/452/vshow

question = """Taken all together, how would you say things are these days--
would you say that you are very happy, pretty happy, or not too happy?
"""

# 1 = very happy
# 2 = pretty happy
# 3 = not too happy

title = "Are you happy?"
subtitle = "Percent saying very happy"
ylim = [10, 45]

In [8]:
value_counts(gss['happy'])

,counts
values,
1.0,21550
2.0,37446
3.0,8681
NaN,4713


In [9]:
missing = gss['happy'].isna()
gss['y1'] = (gss['happy'] == 1).astype(float).mask(missing)
gss['y2'] = (gss['happy'] == 2).astype(float).mask(missing)
gss['y3'] = (gss['happy'] == 3).astype(float).mask(missing)
value_counts(gss['y1'])

,counts
values,
0.0,46127
1.0,21550
NaN,4713


In [10]:
# shift so the codes are 0, 1, 2
data = gss['happy'].values - 1
pd.Series(data).value_counts().sort_index()

0.0    21550
1.0    37446
2.0     8681
Name: count, dtype: int64

In [11]:
gss['cohort'].describe()

count    71987.000000
mean      1991.908095
std        561.717010
min       1883.000000
25%       1938.000000
50%       1954.000000
75%       1968.000000
max       9999.000000
Name: cohort, dtype: float64

In [33]:
bins = [1928, 1946, 1965, 1981, 1997, 2013, 2025] 
generation_labels = ['Silent', 'Boomer', 'GenX', 'Millennial', 'GenZ', 'Alpha']

# Assign each cohort to a generation
gss['generation'] = pd.cut(gss['cohort'], bins=bins, labels=generation_labels, right=False)

In [13]:
# gss_clean = gss.dropna(subset=['generation', 'sex', 'happy']).sample(10000)
gss_clean = gss.dropna(subset=['generation', 'sex', 'happy'])
gss_clean.shape

(56641, 61)

In [14]:
gss_clean['generation'].value_counts()

generation
Boomer        24087
Silent        13348
GenX          12106
Millennial     6049
GenZ           1051
Alpha             0
Name: count, dtype: int64

In [15]:
pd.crosstab(gss_clean['year'], gss_clean['generation'])

generation,Silent,Boomer,GenX,Millennial,GenZ
year,,,,,
1972,484,310,0,0,0
1973,497,327,0,0,0
1974,432,374,0,0,0
1975,470,413,0,0,0
1976,476,431,0,0,0
1977,470,451,0,0,0
1978,444,554,0,0,0
1980,406,569,0,0,0
1982,491,805,0,0,0


In [16]:
# Convert generation to a categorical variable
generation = pd.Categorical(gss_clean['generation'], categories=generation_labels)
generation_codes = generation.codes

In [17]:
years = gss_clean['year'].value_counts().sort_index()
year_labels = years.index.values

In [18]:
# Convert year to a categorical variable
year = pd.Categorical(gss_clean['year'], categories=year_labels)
year_codes = year.codes

In [19]:
# Independent variable (sex)
sex = gss_clean['sex'].values

In [20]:
# Dependent variable (happiness levels)
y = gss_clean['happy'].values - 1
pd.Series(y).value_counts()

1.0    31900
0.0    17384
2.0     7357
Name: count, dtype: int64

## Model 1

In [21]:
# Build the model
with pm.Model() as ordered_logistic_model1:
    
    # Priors for the coefficients
    intercept = pm.Normal('intercept', mu=0, sigma=2)
    beta_sex = pm.Normal('beta_sex', mu=0, sigma=2)
    
    # Priors for the generation coefficients (one per generation)
    beta_gen = pm.Normal('beta_gen', mu=0, sigma=2, shape=len(generation_labels))
    
    # Priors for the cutpoints (thresholds) between categories
    cutpoints = pm.Normal('cutpoints', mu=np.array([-1, 1]), sigma=2, shape=2,
                          transform=pm.distributions.transforms.ordered)
    
    # Linear combination of coefficients and independent variables
    eta = intercept + beta_sex * sex + beta_gen[generation_codes]
    
    # Likelihood: ordered logistic regression
    y_obs = pm.OrderedLogistic('y_obs', eta=eta, cutpoints=cutpoints, 
                               compute_p=False, observed=y)

In [22]:
with ordered_logistic_model1:
    idata = pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [intercept, beta_sex, beta_gen, cutpoints]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 15890 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details


In [23]:
pm.summary(idata)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
intercept,-0.178,1.269,-2.433,2.248,0.034,0.024,1434.0,1806.0,1.00
beta_sex,-0.060,0.017,-0.092,-0.027,0.000,0.000,3013.0,2349.0,1.00
beta_gen[0],-0.418,0.859,-2.092,1.124,0.032,0.023,761.0,1224.0,1.01
beta_gen[1],-0.239,0.859,-1.972,1.241,0.032,0.023,761.0,1262.0,1.01
beta_gen[2],-0.177,0.859,-1.834,1.382,0.032,0.023,762.0,1280.0,1.01
beta_gen[3],-0.013,0.859,-1.671,1.542,0.032,0.022,762.0,1229.0,1.01
beta_gen[4],0.698,0.860,-1.003,2.208,0.032,0.022,757.0,1283.0,1.01
beta_gen[5],-0.040,1.987,-3.912,3.547,0.034,0.044,3506.0,1991.0,1.00
cutpoints[0],-1.318,1.214,-3.678,0.856,0.030,0.022,1582.0,2008.0,1.00
cutpoints[1],1.417,1.213,-0.912,3.623,0.030,0.022,1583.0,2008.0,1.00


In [24]:
import arviz as az

az.to_netcdf(idata, "ordered_logistic_model1.nc")

'ordered_logistic_model1.nc'

In [25]:
!ls -lh ordered_logistic_model1.nc

-rw-rw-r-- 1 downey downey 740K Aug 30 01:22 ordered_logistic_model1.nc


## Model 2

In [26]:
# Build the model
with pm.Model() as ordered_logistic_model2:
    
    # Priors for the coefficients
    intercept = pm.Normal('intercept', mu=0, sigma=2)
    
    # Priors for the generation coefficients (one per generation)
    beta_gen = pm.Normal('beta_gen', mu=0, sigma=2, shape=len(generation_labels))
    
    # Priors for the generation coefficients (one per generation)
    beta_year = pm.Normal('beta_year', mu=0, sigma=2, shape=len(year_labels))
    
    # Priors for the cutpoints (thresholds) between categories
    cutpoints = pm.Normal('cutpoints', mu=np.array([-1, 1]), sigma=2, shape=2,
                          transform=pm.distributions.transforms.ordered)
    
    # Linear combination of coefficients and independent variables
    eta = intercept + beta_year[year_codes] + beta_gen[generation_codes]
    
    # Likelihood: ordered logistic regression
    y_obs = pm.OrderedLogistic('y_obs', eta=eta, cutpoints=cutpoints, 
                               compute_p=False, observed=y)

In [27]:
with ordered_logistic_model2:
    idata = pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [intercept, beta_gen, beta_year, cutpoints]


Output()

/home/downey/miniconda3/envs/SurveyDataPyMC/lib/python3.12/site-packages/pytensor/compile/function/types.py:970: RuntimeWarning: invalid value encountered in accumulate
  self.vm()


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 9762 seconds.
Chain 3 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


In [28]:
pm.summary(idata)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
intercept,-0.383,1.292,-2.851,2.034,0.073,0.052,313.0,600.0,1.01
beta_gen[0],-0.263,0.906,-1.969,1.383,0.068,0.048,179.0,375.0,1.01
beta_gen[1],-0.096,0.905,-1.749,1.593,0.068,0.048,180.0,362.0,1.01
beta_gen[2],-0.047,0.905,-1.696,1.643,0.068,0.048,180.0,388.0,1.01
beta_gen[3],-0.001,0.904,-1.587,1.750,0.067,0.048,180.0,358.0,1.01
beta_gen[4],0.478,0.906,-1.129,2.203,0.067,0.048,181.0,386.0,1.01
beta_gen[5],-0.077,1.928,-3.934,3.408,0.070,0.051,767.0,1020.0,1.00
beta_year[0],0.152,0.326,-0.446,0.777,0.036,0.025,84.0,224.0,1.04
beta_year[1],-0.024,0.324,-0.625,0.583,0.036,0.025,83.0,179.0,1.04
beta_year[2],-0.087,0.325,-0.720,0.501,0.036,0.026,82.0,212.0,1.04


In [29]:
import arviz as az

az.to_netcdf(idata, "ordered_logistic_model2.nc")

'ordered_logistic_model2.nc'

In [30]:
!ls -lh ordered_logistic_model2.nc

-rw-rw-r-- 1 downey downey 1.7M Aug 30 04:05 ordered_logistic_model2.nc


## Model 3

With a different parameter for each year, generation pair

In [34]:

with pm.Model() as interaction_model:
    
    # Priors for the global intercept
    intercept = pm.Normal('intercept', mu=0, sigma=2)
    
    # Hyperpriors for the group-level means and standard deviations
    mu_gen_year = pm.Normal('mu_gen_year', mu=0, sigma=2)
    sigma_gen_year = pm.HalfNormal('sigma_gen_year', sigma=2)
    
    # Group-level effects with hierarchical structure: a coefficient for each year-generation combination
    beta_gen_year = pm.Normal('beta_gen_year', mu=mu_gen_year, sigma=sigma_gen_year, 
                              shape=(len(year_labels), len(generation_labels)))
    
    # Priors for the cutpoints (thresholds) between categories
    cutpoints = pm.Normal('cutpoints', mu=np.array([-1, 1]), sigma=2, shape=2,
                          transform=pm.distributions.transforms.ordered)
    
    # Linear combination of coefficients and independent variables
    eta = intercept + beta_gen_year[year_codes, generation_codes]
    
    # Likelihood: ordered logistic regression
    y_obs = pm.OrderedLogistic('y_obs', eta=eta, cutpoints=cutpoints, 
                               compute_p=False, observed=y)

In [35]:
with interaction_model:
    idata = pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [intercept, mu_gen_year, sigma_gen_year, beta_gen_year, cutpoints]


Output()

/home/downey/miniconda3/envs/SurveyDataPyMC/lib/python3.12/site-packages/pytensor/compile/function/types.py:970: RuntimeWarning: invalid value encountered in accumulate
  self.vm()
/home/downey/miniconda3/envs/SurveyDataPyMC/lib/python3.12/site-packages/pytensor/compile/function/types.py:970: RuntimeWarning: invalid value encountered in accumulate
  self.vm()
/home/downey/miniconda3/envs/SurveyDataPyMC/lib/python3.12/site-packages/pytensor/compile/function/types.py:970: RuntimeWarning: invalid value encountered in accumulate
  self.vm()
/home/downey/miniconda3/envs/SurveyDataPyMC/lib/python3.12/site-packages/pytensor/compile/function/types.py:970: RuntimeWarning: invalid value encountered in accumulate
  self.vm()


Sampling 4 chains for 1_000 tune and 152 draw iterations (4_000 + 608 draws total) took 9411 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


In [40]:
pm.summary(idata)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
intercept,-0.418,1.246,-2.395,2.073,0.212,0.151,34.0,330.0,1.10
mu_gen_year,-0.052,0.545,-0.830,0.908,0.262,0.199,5.0,12.0,2.97
"beta_gen_year[0, 0]",-0.063,0.550,-0.805,0.947,0.261,0.199,5.0,13.0,2.73
"beta_gen_year[0, 1]",0.126,0.554,-0.683,1.169,0.261,0.198,5.0,14.0,2.66
"beta_gen_year[0, 2]",-0.031,0.625,-0.976,1.279,0.266,0.199,6.0,35.0,1.91
...,...,...,...,...,...,...,...,...,...
"beta_gen_year[33, 4]",0.423,0.552,-0.308,1.487,0.262,0.199,5.0,12.0,2.79
"beta_gen_year[33, 5]",-0.036,0.627,-1.073,1.150,0.257,0.191,6.0,55.0,1.75
sigma_gen_year,0.305,0.025,0.266,0.354,0.002,0.001,250.0,296.0,1.01
cutpoints[0],-1.303,1.168,-3.335,0.798,0.063,0.044,348.0,391.0,1.02


In [37]:
import arviz as az

az.to_netcdf(idata, "interaction_model.nc")

'interaction_model.nc'

In [38]:
!ls -lh interaction_model.nc

-rw-rw-r-- 1 downey downey 1.2M Aug 30 09:16 interaction_model.nc


In [39]:
idata = az.from_netcdf("interaction_model.nc")

## Model 4

Hierarchical interaction model

In [ ]:
with pm.Model() as hierarchical_model:
    
    # Priors for the global intercept
    intercept = pm.Normal('intercept', mu=0, sigma=2)
    
    # Top-level: Hyperpriors for the year-level effects
    mu_year = pm.Normal('mu_year', mu=0, sigma=2)
    sigma_year = pm.HalfNormal('sigma_year', sigma=2)
    beta_year = pm.Normal('beta_year', mu=mu_year, sigma=sigma_year, shape=len(year_labels))
    
    # Second-level: Hyperpriors for generation effects within each year
    mu_gen = pm.Normal('mu_gen', mu=0, sigma=2, shape=len(year_labels))
    sigma_gen = pm.HalfNormal('sigma_gen', sigma=2, shape=len(year_labels))
    beta_gen = pm.Normal('beta_gen', mu=mu_gen[:, None], sigma=sigma_gen[:, None], 
                         shape=(len(year_labels), len(generation_labels)))
    
    # Priors for the cutpoints (thresholds) between categories
    cutpoints = pm.Normal('cutpoints', mu=np.array([-1, 1]), sigma=2, shape=2,
                          transform=pm.distributions.transforms.ordered)
    
    # Linear combination of coefficients and independent variables
    eta = intercept + beta_year[year_codes] + beta_gen[year_codes, generation_codes]
    
    # Likelihood: ordered logistic regression
    y_obs = pm.OrderedLogistic('y_obs', eta=eta, cutpoints=cutpoints, 
                               compute_p=False, observed=y)

In [ ]:
with hierarchical_model:
    idata = pm.sample()

In [ ]:
pm.summary(idata)

In [ ]:
import arviz as az

az.to_netcdf(idata, "hierarchical_model.nc")

In [ ]:
!ls -lh hierarchical_model.nc

In [ ]:
idata = az.from_netcdf("hierarchical_model.nc")